|Purpose/Summary|
|-|
|basic ticker symbol, cik, and company name data, for all SEC companies that filed with the SEC since 1994.|

In [22]:
import requests
import pandas as pd
import json

In [23]:
# get "all-time" basic stock ticker data from SEC server endpoint
def get_basic_ticker_listing_data() -> pd.DataFrame:
    headers = {
        'User-Agent': 'ServiceNow fred.luddy@servicenow.com', 
        'Accept': 'application/json',
        'Host': 'www.sec.gov'
    }

    try:
        response = requests.get(
            'https://www.sec.gov/files/company_tickers.json',
            headers=headers
        )
        
        if response.status_code == 403:
            raise Exception("SEC requires a proper User-Agent. \nPlease modify the User-Agent header with your company/contact information.")
        
        response.raise_for_status()

        # print(json.dumps(response.json(), indent=4))
        
        df = pd.DataFrame.from_dict(response.json(), orient='index')
        df['cik_str'] = df['cik_str'].astype(str).str.zfill(10)
        return df
        
    except requests.exceptions.RequestException as e:
        raise Exception(f"Error accessing SEC API: {str(e)}")

In [24]:
basic_ticker_listing_df = get_basic_ticker_listing_data()

In [25]:
# display(sec_all_tickers_df)
print(len(basic_ticker_listing_df))
print(basic_ticker_listing_df.columns)
print(basic_ticker_listing_df.head(3))

9998
Index(['cik_str', 'ticker', 'title'], dtype='object')
      cik_str ticker           title
0  0001045810   NVDA     NVIDIA CORP
1  0000320193   AAPL      Apple Inc.
2  0000789019   MSFT  MICROSOFT CORP


In [26]:
# save to .csv (ready for data pipeline)
basic_ticker_listing_df.to_csv('DATA_PIPELINE_READY/basic_ticker_listing_data_94.csv', index=False)